In [ ]:
!pip install ipycanvas
!pip install ipyevents
!pip install Pillow
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.7 MB/s eta 0:00:00


In [ ]:
# for model
import tensorflow as tf
import keras
from keras import layers

# gui
import ipycanvas
import ipyevents

In [ ]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
model = keras.Sequential(
    [
        layers.Flatten(input_shape=(28,28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ]
)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,386 (427.29 KB)

 Trainable params: 109,386 (427.29 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=32
)

Epoch 1/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - accuracy: 0.8723 - loss: 0.4457 - val_accuracy: 0.9678 - val_loss: 0.1127
Epoch 2/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9658 - loss: 0.1123 - val_accuracy: 0.9678 - val_loss: 0.1052
Epoch 3/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9759 - loss: 0.0783 - val_accuracy: 0.9753 - val_loss: 0.0796
Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9824 - loss: 0.0559 - val_accuracy: 0.9762 - val_loss: 0.0782
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9858 - loss: 0.0440 - val_accuracy: 0.9778 - val_loss: 0.0812
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9899 - loss: 0.0326 - val_accuracy: 0.9798 - val_loss: 0.0792
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9910 - loss: 0.0261 - val_accuracy: 0.9790 - val_loss: 0.0852
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9917 - loss: 0.0237 

In [ ]:
predictions = model.predict(x_test[:30])
print(predictions.argmax(axis=1))
print(y_test[:30])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1]


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import numpy as np
from ipycanvas import Canvas, hold_canvas
from ipywidgets import Button, HBox, VBox, Label, Output, IntSlider
from IPython.display import display
import matplotlib.pyplot as plt
import time

class MNISTDrawingCanvas:
    def __init__(self, model):
        self.model = model

        self.canvas_size = 28
        self.canvas = Canvas(width=self.canvas_size, height=self.canvas_size, sync_image_data=True)
        self.canvas.layout.width = '140px'
        self.canvas.layout.height = '140px'
        self.canvas.fill_style = 'white'
        self.canvas.fill_rect(0, 0, self.canvas_size, self.canvas_size)
        self.canvas.stroke_style = 'black'
        self.canvas.line_width = 2
        self.canvas.line_cap = 'round'
        self.canvas.line_join = 'round'

        self.drawing = False
        self.last_x = 0
        self.last_y = 0
        self.brush_slider = IntSlider(value=2, min=1, max=4, step=1,
                                     description='Brush:', continuous_update=True)
        self.brush_slider.observe(self.update_brush_size, names='value')

        self.clear_btn = Button(description='Clear', button_style='warning', icon='trash')
        self.predict_btn = Button(description='Predict', button_style='success', icon='check')

        self.prediction_label = Label(value='Draw a digit and click Predict')
        self.output = Output()

        self.canvas.on_mouse_down(self.on_mouse_down)
        self.canvas.on_mouse_move(self.on_mouse_move)
        self.canvas.on_mouse_up(self.on_mouse_up)
        self.clear_btn.on_click(self.clear_canvas)
        self.predict_btn.on_click(self.predict_digit)

        controls = HBox([self.clear_btn, self.predict_btn, self.brush_slider])
        self.widget = VBox([
            self.canvas,
            controls,
            self.prediction_label,
            self.output
        ])

    def update_brush_size(self, change):
        self.canvas.line_width = change['new']

    def on_mouse_down(self, x, y):
        self.drawing = True
        self.last_x = x
        self.last_y = y
        with hold_canvas(self.canvas):
            self.canvas.begin_path()
            self.canvas.arc(x, y, self.canvas.line_width/2, 0, 2 * np.pi)
            self.canvas.fill()

    def on_mouse_move(self, x, y):
        if self.drawing:
            with hold_canvas(self.canvas):
                self.canvas.begin_path()
                self.canvas.move_to(self.last_x, self.last_y)
                self.canvas.line_to(x, y)
                self.canvas.stroke()
            self.last_x = x
            self.last_y = y

    def on_mouse_up(self, x, y):
        self.drawing = False

    def clear_canvas(self, btn):
        self.canvas.clear()
        self.canvas.fill_style = 'white'
        self.canvas.fill_rect(0, 0, self.canvas_size, self.canvas_size)
        self.prediction_label.value = 'Draw a digit and click Predict'
        self.output.clear_output()

    def get_mnist_array(self):
      # Force drawing operations to send to browser
      self.canvas.flush()

      # Give browser 1 animation frame to sync image_data
      time.sleep(0.05)

      # Now read pixel data (this method always exists)
      img = self.canvas.get_image_data(0, 0, self.canvas_size, self.canvas_size)

      # Convert to numpy
      img_data = np.array(img)

      # Convert RGBA → grayscale
      gray = np.mean(img_data[:, :, :3], axis=2)

      # MNIST expects white digit on black
      inverted = 255 - gray

      return inverted / 255.0


    def predict_digit(self, btn):
        """Predict and show results"""
        img_array = self.get_mnist_array()
        model_input = img_array.reshape(1, 28, 28)

        try:
            predictions = self.model.predict(model_input, verbose=0)
            probs = predictions[0]
            predicted_digit = np.argmax(probs)
            confidence = probs[predicted_digit]

            self.prediction_label.value = f'Predicted: {predicted_digit} ({confidence:.1%})'

            with self.output:
                self.output.clear_output(wait=True)
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))

                # Processed image
                ax1.imshow(img_array, cmap='gray')
                ax1.set_title('28×28 Input')
                ax1.axis('off')

                # Probability bar chart
                colors = ['green' if i == predicted_digit else 'lightgray' for i in range(10)]
                ax2.bar(range(10), probs, color=colors)
                ax2.set_xlabel('Digit')
                ax2.set_ylabel('Probability')
                ax2.set_ylim([0, 1])

                plt.tight_layout()
                plt.show()

        except Exception as e:
            self.prediction_label.value = f'Error: {str(e)}'

    def display(self):
        display(self.widget)


# Create and display the canvas
drawing_canvas = MNISTDrawingCanvas(model)
drawing_canvas.display()